## QF620 Project

##  Part I (Analytical Option Formula)

Consider the following European options: 

- Vanilla call/put 
- Digital cash-or-nothing call/put 
- Digital asset-or-nothing call/put

Derive and implement the following models to value these options in Python: 

- 1 Black-Scholes model 
- 2 Bachelier model 
- 3 Black76 model 
- 4 Displaced-diﬀusion model

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from math import log, sqrt, exp
from scipy.optimize import least_squares
%matplotlib inline

### Vanilla call

#### Black-Scholes model

In [10]:
#正常Black Scholes
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [17]:
#Stock as numeraire
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r - sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-r * T) * norm.cdf(d1) - K * np.exp(sigma**2*T) * norm.cdf(d2)

In [19]:
BlackScholesCall(100, 100, 0.01, 0.1, 1)

3.987761167674492

In [21]:
change = 0.001
a = BlackScholesCall(100+change, 100, 0.03, 0.2, 1)
b = BlackScholesCall(100-change, 100, 0.03, 0.2, 1)
delta = (a-b)/(change*2)
delta

0.5987063256149838

In [25]:
change = 0.001
a = BlackScholesCall(100, 100, 0.03, 0.2*(1+change), 1)
b = BlackScholesCall(100, 100, 0.03, 0.2*(1-change), 1)
vega = (a-b)/(0.2*change*2)
vega

38.666811181862215

In [27]:
change = 0.001
a = BlackScholesCall(100, 100, 0.03, 0.2, 1)
b = BlackScholesCall(100, 100, 0.03, 0.2, 1-change)
theta = (b-a)/(change)
theta

-5.381433838373084

In [16]:
def BlackScholesCallDelta(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return norm.cdf(d1)

In [7]:
def BlackScholesCalld1(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1

In [17]:
def BlackScholesCallNd2(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return norm.cdf(d2)

#### Bachelier model

In [3]:
def BachelierCall(S,K,r,sigma,T):
    d1=(S-K) / (S * sigma * np.sqrt(T))
    return (S-K) * norm.cdf(d1) + S * sigma * np.sqrt(T) * norm.pdf(d1)

#### Black76 model 

In [31]:
def Black76Call(F,K,r,sigma,T):
    d1= (np.log(F / K )+(1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2=d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))

In [32]:
def Black76Put(F,K,r,sigma,T):
    d1 = (np.log(F / K) + (1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))

In [33]:
Black76Call(100,100,0.01,0.1,1)

3.9480822810875194

In [34]:
Black76Put(100,100,0.01,0.1,1)

3.9480822810875194

#### Displaced-diﬀusion model

In [5]:
def Displaced_diffusionCall(F,K,r,sigma,T,beta):
    Fd = F / beta
    sigmad = sigma * beta
    Kd = K + ((1-beta) / beta) * F
    d1 = (np.log(Fd / Kd) + (sigmad ** 2 / 2) * T) / (sigmad * np.sqrt(T))
    d2 = d1 - sigmad * np.sqrt(T)
    return np.exp(-r * T) * (Fd * norm.cdf(d1) - Kd * norm.cdf(d2))

### Vanilla put

#### Black-Scholes model

In [29]:
def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

#### Bachelier model

In [7]:
def BachelierPut(S,K,r,sigma,T):
    d1 = (S-K) / (S * sigma * np.sqrt(T))
    return (K-S) * norm.cdf(-d1) + S *sigma * np.sqrt(T) * norm.pdf(-d1)

#### Black76 model 

In [8]:
def Black76Put(F,K,r,sigma,T):
    d1 = (np.log(F / K) + (1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))

#### Displaced-diﬀusion model

In [9]:
def Displaced_diffusionPut(F,K,r,sigma,T,beta):
    Fd = F / beta
    sigmad = sigma * beta
    Kd = K + ((1 - beta) / beta) * F
    d1 = (np.log(Fd / Kd) + (sigmad ** 2 / 2) * T) / (sigmad * np.sqrt(T))
    d2 = d1 - sigmad * np.sqrt(T)
    return np.exp(-r * T) * (Kd * norm.cdf(-d2) - Fd * norm.cdf(-d1))

### Digital cash-or-nothing call

#### Black-Scholes model

In [10]:
def BlackScholesCall_Di_Ca(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(d2)

#### Bachelier model

In [11]:
def BachelierCall_Di_Ca(S,K,r,sigma,T):
    d1=(S-K) / (S * sigma * np.sqrt(T))
    return norm.cdf(d1)

#### Black76 model 

In [12]:
def Black76Call_Di_Ca(F,K,r,sigma,T):
    d1 = (np.log(F / K )+(1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(d2)

#### Displaced-diﬀusion model

In [13]:
def Displaced_diffusionCall_Di_Ca(F,K,r,sigma,T,beta):
    Fd = F / beta
    sigmad = sigma * beta
    Kd = K + ((1-beta) / beta) * F
    d1 = (np.log(Fd / Kd) + (sigmad ** 2 / 2) * T) / (sigmad * np.sqrt(T))
    d2 = d1 - sigmad * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(d2)

### Digital cash-or-nothing put

#### Black-Scholes model

In [14]:
def BlackScholesPut_Di_Ca(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2)

#### Bachelier model

In [15]:
def BachelierPut_Di_Ca(S,K,r,sigma,T):
    d1 = (S-K) / (S * sigma * np.sqrt(T))
    return norm.cdf(-d1)

#### Black76 model 

In [16]:
def Black76Put_Di_Ca(F,K,r,sigma,T):
    d1 = (np.log(F / K) + (1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2)

#### Displaced-diﬀusion model

In [17]:
def Displaced_diffusionPut_Di_Ca(F,K,r,sigma,T,beta):
    Fd = F / beta
    sigmad = sigma * beta
    Kd = K + ((1 - beta) / beta) * F
    d1 = (np.log(Fd / Kd) + (sigmad ** 2 / 2) * T) / (sigmad * np.sqrt(T))
    d2 = d1 - sigmad * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2)

### Digital asset-or-nothing call

#### Black-Scholes model

In [18]:
def BlackScholesCall_Di_As(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1)

#### Bachelier model

In [19]:
def BachelierCall_Di_As(S,K,r,sigma,T):
    d1=(S-K) / (S * sigma * np.sqrt(T))
    return S * norm.cdf(d1) + S * sigma * np.sqrt(T) * norm.pdf(d1)

#### Black76 model 

In [21]:
def Black76Call_Di_As(F,K,r,sigma,T):
    d1= (np.log(F / K )+(1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2=d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * F * norm.cdf(d1)

#### Displaced-diﬀusion model

In [22]:
def Displaced_diffusionCall_Di_As(F,K,r,sigma,T,beta):
    Fd = F / beta
    sigmad = sigma * beta
    Kd = K + ((1-beta) / beta) * F
    d1 = (np.log(Fd / Kd) + (sigmad ** 2 / 2) * T) / (sigmad * np.sqrt(T))
    d2 = d1 - sigmad * np.sqrt(T)
    return np.exp(-r * T) * Fd * norm.cdf(d1)

### Digital asset-or-nothing put

#### Black-Scholes model

In [23]:
def BlackScholesPut_Di_As(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(-d1)

#### Bachelier model

In [24]:
def BachelierPut_Di_As(S,K,r,sigma,T):
    d1 = (S-K) / (S * sigma * np.sqrt(T))
    return S * norm.cdf(-d1) + S *sigma * np.sqrt(T) * norm.pdf(-d1)

#### Black76 model 

In [25]:
def Black76Put_Di_As(F,K,r,sigma,T):
    d1 = (np.log(F / K) + (1 / 2) * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) *  F * norm.cdf(-d1)

#### Displaced-diﬀusion model

In [26]:
def Displaced_diffusionPut_Di_As(F,K,r,sigma,T,beta):
    Fd = F / beta
    sigmad = sigma * beta
    Kd = K + ((1 - beta) / beta) * F
    d1 = (np.log(Fd / Kd) + (sigmad ** 2 / 2) * T) / (sigmad * np.sqrt(T))
    d2 = d1 - sigmad * np.sqrt(T)
    return np.exp(-r * T) * Fd * norm.cdf(-d1)